# Extract layers for step 1

This notebook includes the code needed to extract layers for Step 1 from different sources.

## Set up

### Library import

In [ ]:
import os
import sys
from glob import glob
from pathlib import Path

import geopandas as gpd
import rasterio as rio
from rasterio.merge import merge

sys.path.append("../src")

from data_processing.raster_processor import RasterProcessor
from data_processing.utils import clip_raster_to_country_and_create_cog

## Data retrieval and processing

### [WorldCover](https://github.com/ESA-WorldCover/esa-worldcover-datasets)

In [ ]:
# Download WorldCover for a country
!python ../scripts/download.py -c Spain -o ../data/raw/WorldCover

### [World Settlement Footprint (WSF) Bulding Area](https://download.geoservice.dlr.de/WSF3D/files/global/WSF3D_V02_BuildingArea.tif)

In [ ]:
# Clip WSF Building Area raster to country and create COG
raster_file = "../data/processed/WSF/WSF3D_V02_BuildingArea.tif"
countries = [
    "Uzbekistan",
]

cog_files = []

for country in countries:
    cog_file = clip_raster_to_country_and_create_cog(raster_file, country)
    if cog_file:
        cog_files.append(cog_file)
print(f"Created {len(cog_files)} COG files: {cog_files}")

In [ ]:
# WSF Building Area
country = "Uzbekistan"
print(f"Processing WSF Building Area for {country}...")

BASE_PATH = Path("../data/processed/WSF/")
input_file = BASE_PATH / f"WSF3D_V02_BuildingArea_{country.replace(' ', '_')}.tif"
style_file = BASE_PATH / "WSF3D_V02_BuildingArea_satellite.qml"
output_file = Path(BASE_PATH / f"Rasters/WSF3D_V02_BuildingArea_{country.replace(' ', '_')}.tif")  # noqa: E501
layer_name = f"{country.replace(' ', '_')}_WSF3D_BuildingArea"

RasterProcessor(
    input_file,
    output_file=output_file,
    qml_file=style_file,
    layer_name=layer_name,
    upload=False,
    create_mbtiles=False,
).process()

In [ ]:
# Merge WSF Building Area COGs when needed (e.g., for West Africa)
countries = [
    "Mali",
    "Niger",
    "Burkina Faso",
    "Nigeria",
    "Benin",
    "Togo",
    "Ghana",
    "Côte d'Ivoire",

]
# Directory containing your .tif files
raster_dir = "../data/processed/WSF/"

# Collect matching raster paths
raster_files = []
for country in countries:
    country_filename = country.replace(" ", "_")
    pattern = os.path.join(raster_dir, f"*_{country_filename}.tif")  # <-- underscore added
    matched = glob(pattern)
    raster_files.extend(matched)

# Optional: deduplicate in case of edge cases
raster_files = list(dict.fromkeys(raster_files))
print(f"Found {len(raster_files)} raster files to merge.")

if not raster_files:
    raise FileNotFoundError("No matching raster files found. Check filenames and directory path.")

# Read and merge rasters
src_files_to_mosaic = [rio.open(fp) for fp in raster_files]
mosaic, out_trans = merge(src_files_to_mosaic)

# Use metadata from first file as template
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans,
    "compress": "DEFLATE",
    "tiled": True,
    "blockxsize": 256,
    "blockysize": 256,
    "nodata": 0
})

# Save merged output
output_path = os.path.join(raster_dir, "WSF3D_V02_BuildingArea_Western_Africa.tif")
with rio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)

print(f"Merged raster saved to: {output_path}")

In [ ]:
# Create subset of GADM for a group of countries (e.g., West Africa)
countries = [
    "Mali",
    "Niger",
    "Burkina Faso",
    "Nigeria",
    "Benin",
    "Togo",
    "Ghana",
    "Côte d'Ivoire",
]

# Load the full GADM shapefile
gadm_file = Path("../data/raw/gadm_410-adm_0/gadm_410-adm_0.shp")
gdf_full = gpd.read_file(gadm_file)

# Filter to only your countries of interest
west_africa_gdf = gdf_full[gdf_full["COUNTRY"].isin(countries)].copy()

print(f"Original shapefile: {len(gdf_full)} countries")
print(f"Filtered subset: {len(west_africa_gdf)} countries")
print(f"Countries found: {west_africa_gdf['COUNTRY'].tolist()}")

# Check if any countries are missing
missing_countries = set(countries) - set(west_africa_gdf['COUNTRY'].tolist())
if missing_countries:
    print(f"Countries not found in GADM: {missing_countries}")

# Create output directory and save the subset shapefile
output_shapefile = Path("../data/raw/Water/West Africa/boundaries/west_africa_countries.shp")
output_shapefile.parent.mkdir(parents=True, exist_ok=True)

# Save the filtered subset
west_africa_gdf.to_file(output_shapefile)
print(f"West Africa subset saved to: {output_shapefile}")
